# Sequence query

In [22]:
import asyncio
import textwrap
from typing import Annotated, Generic, TypeVar

from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI

import pandas as pd


chat_model = ChatOpenAI(model="gpt-4o-mini")

messages: list[BaseMessage] = [
    SystemMessage(
        content="""# Identity

* You are an AI assistant."""
    ),
    HumanMessage(
        content="Tôi có một prompt template, hãy tạo ra những query để test cho prompt template đó."
        "Phải báo gồm các truy vấn.\n"
        "* Truy vấn hợp lệ\n"
        "* Truy vấn biên\n"
        "* Truy vấn định dạng không chuẩn\n"
        "* Truy vấn không hợp lệ\n"
        "* Truy vấn gây nhiễu."
    ),
]

response = await chat_model.ainvoke(messages)
messages.append(response)

instruction = input("Enter your instruction")


class ReasoningResponse(BaseModel):
    reasoning: Annotated[
        str,
        Field(
            description="Giải thích chi tiết quá trình xử lý hoặc lý do đầu vào không hợp lệ"
        ),
    ]
    errorMessages: Annotated[
        list[str], Field(description="Array or error messages (empty if none)")
    ]


class ResultResponse(ReasoningResponse):
    result: int | None


prompt_template = f"""# Identity

* You are an AI assistant.

# Instructions

* {instruction}
* {textwrap.indent(PydanticOutputParser(pydantic_object=ResultResponse).get_format_instructions(), "  ", lambda line: True).strip()}"""

messages.append(HumanMessage(content=f"Đây là prompt template: {prompt_template}"))

response = await chat_model.ainvoke(messages)
messages.append(response)


class Response(ReasoningResponse):
    queries: Annotated[list[str], Field(description="List of queries to OpenAI.")]


output_parser = PydanticOutputParser(pydantic_object=Response)
messages.append(
    HumanMessage(
        content=f"""* Parse all user queries of the previous step.
* {output_parser.get_format_instructions()}"""
    )
)

response = await chat_model.ainvoke(messages)
messages.append(response)

model = output_parser.invoke(response)

tasks = []
for user_query in model.queries:
    tasks.append(
        chat_model.ainvoke(
            [
                SystemMessage(content=prompt_template),
                HumanMessage(content=user_query),
            ]
        )
    )


responses = await asyncio.gather(*tasks, return_exceptions=True)

data_frame = pd.DataFrame(
    zip(model.queries, [response.text for response in responses]),
    columns=["User prompt", "Response"],
)

data_frame

,User prompt,Response
0,Tính tổng của 5 và 10.,"{\n ""reasoning"": ""Tổng của 5 và 10 là 15, đượ..."
1,Tính tổng của -3 và 7.,"{\n ""reasoning"": ""Tổng của -3 và 7 được tính ..."
2,Tính tổng của số lớn nhất và nhỏ nhất có thể (...,"```json\n{\n ""reasoning"": ""Tổng của số lớn nh..."
3,"5 và 10, tính tổng nhé!","{\n ""reasoning"": ""Tổng của hai số 5 và 10 đượ..."
4,Tính tổng của hai chữ cái A và B.,"```json\n{\n ""reasoning"": ""Không thể tính tổn..."
5,Tính tổng của 7 và 'bánh mì'?,"```json\n{\n ""reasoning"": ""Một trong các đầu ..."
6,Tổng của hai số bằng có thể là gì?,"Để tính tổng của hai số, bạn chỉ cần cộng chún..."
